In [1]:
import numpy
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
# import pycountry
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
import numba
import copy
import glob
from numba import njit
import multiprocessing
from functools import partial

---

In [2]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {
                   'variable': ["temperature"],
                   'date': ['20151229','20151230'],
                   'bbox': [40,10,50,21],
                   'hdf':'True',
               })
r.download(target='download_bbox.nc')
cdsfile = glob.glob('./download_bbox.nc')
cdsdata=eua.CDMDataset(cdsfile[0])
cdsdata

2021-07-22 15:29:19,554 INFO Welcome to the CDS
2021-07-22 15:29:19,557 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-07-22 15:29:19,801 INFO Request is completed
2021-07-22 15:29:19,803 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.comprehensive_upper_air.retrieve-1626959191.9041104-25948-8-73be67c8-070e-41e8-b52d-cfa72e4431f1.zip to download_bbox.nc (4.1M)
2021-07-22 15:29:21,241 INFO Download rate 2.8M/s   


File: <HDF5 file "download_bbox.nc" (mode r)>
Permissions: RW (mblaschek)
Filesize: 4.06 MB
Filename: ./download_bbox.nc
(G)roups/(V)ariables: 

 - G | 0-20000-0-10868______________________________ : : 13
 - G | 0-20000-0-10954______________________________ : : 13
 - G | 0-20000-0-11010______________________________ : : 13
 - G | 0-20000-0-11120______________________________ : : 13
 - G | 0-20000-0-11240______________________________ : : 13
 - G | 0-20000-0-11747______________________________ : : 13
 - G | 0-20000-0-11952______________________________ : : 13
 - G | 0-20000-0-12843______________________________ : : 13
 - G | 0-20000-0-12982______________________________ : : 13
 - G | 0-20000-0-13275______________________________ : : 13
 - G | 0-20000-0-14015______________________________ : : 13
 - G | 0-20000-0-14240______________________________ : : 13
 - G | 0-20000-0-14430______________________________ : : 13
 - G | 0-20000-0-16144______________________________ : : 13
 - G | 0-20001-

In [3]:
cdsdata['0-20001-0-11035']

0-20001-0-11035:

lat_______________________________________________ : : (7425,)
lon_______________________________________________ : : (7425,)
obs_______________________________________________ : : (7425,)
plev______________________________________________ : : (7425,)
report_id_________________________________________ : : (7425, 11)
station_id________________________________________ : : (7425, 25)
string11__________________________________________ : : (11,)
string25__________________________________________ : : (25,)
ta________________________________________________ : : (7425,)
time______________________________________________ : : (7425,)
trajectory________________________________________ : : (4,)
trajectory_index__________________________________ : : (7425,)
trajectory_label__________________________________ : : (4, 11)

In [4]:
cdsdata.to_dataframe(groups=['0-20001-0-11035'], variables=['ta', 'time', 'plev'])

2021-07-22 15:29:21,738 INFO Reading Profile on slice(None, None, None)


,ta,time,plev
0,237.919998,3660377437,640.0
1,237.320007,3660377437,650.0
2,237.100006,3660377437,660.0
3,236.320007,3660377437,670.0
4,234.360001,3660377437,680.0
...,...,...,...
7420,270.049988,3660507057,100080.0
7421,270.230011,3660507057,100290.0
7422,270.390015,3660507057,100520.0
7423,270.500000,3660507057,100720.0


In [5]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {
                   'variable': ["temperature"],
                   'date': ['20151229','20151230'],
                   'statid': ['11035', '10393','07257'],
                   'hdf':'True',
               })
r.download(target='download_statid.nc')
cdsfile = glob.glob('./download_statid.nc')
cdsdata=eua.CDMDataset(cdsfile[0])
cdsdata

2021-07-22 15:29:21,936 INFO Welcome to the CDS
2021-07-22 15:29:21,938 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-07-22 15:29:22,104 INFO Request is completed
2021-07-22 15:29:22,147 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.comprehensive_upper_air.retrieve-1626959377.8795846-29613-6-38a766d2-b683-4ef7-bdf9-7ee09b5f091c.zip to download_statid.nc (960.3K)
2021-07-22 15:29:22,612 INFO Download rate 2M/s   


File: <HDF5 file "download_statid.nc" (mode r)>
Permissions: RW (mblaschek)
Filesize: 0.94 MB
Filename: ./download_statid.nc
(G)roups/(V)ariables: 

 - G | 0-20001-0-10393______________________________ : : 13
 - G | 0-20001-0-11035______________________________ : : 13

In [6]:
cdsdata['0-20001-0-10393']

0-20001-0-10393:

lat_______________________________________________ : : (19370,)
lon_______________________________________________ : : (19370,)
obs_______________________________________________ : : (19370,)
plev______________________________________________ : : (19370,)
report_id_________________________________________ : : (19370, 11)
station_id________________________________________ : : (19370, 20)
string11__________________________________________ : : (11,)
string20__________________________________________ : : (20,)
ta________________________________________________ : : (19370,)
time______________________________________________ : : (19370,)
trajectory________________________________________ : : (8,)
trajectory_index__________________________________ : : (19370,)
trajectory_label__________________________________ : : (8, 11)

In [7]:
cdsdata.close()

## parallel access

In [8]:
cdsdata.groups

['0-20001-0-10393', '0-20001-0-11035']

In [9]:
def testfunc(file, group):
    with h5py.File(file, 'r') as data:
        print(data[group]['ta'][:])
    return 0

stations = cdsdata.groups
file = cdsfile[0]
pool = multiprocessing.Pool(processes=2)
func=partial(testfunc, file)
result_list = list(pool.map(func, stations))

[204.06 204.2  204.34 ... 269.85 269.84 269.77][237.92 237.32 237.1  ... 270.39 270.5  270.65]



In [10]:
def testfunc(file, group):
    with h5py.File(file, 'r') as data:
        for i in data[group]['ta'][:100]:
            print(group, i)
    return 0

stations = cdsdata.groups
file = cdsfile[0]
pool = multiprocessing.Pool(processes=2)
func=partial(testfunc, file)
result_list = list(pool.map(func, stations))

0-20001-0-10393 0-20001-0-11035204.06 
237.920-20001-0-10393
 0-20001-0-11035204.2 
237.320-20001-0-10393
 0-20001-0-11035204.34 
237.10-20001-0-10393
 204.510-20001-0-11035
 0-20001-0-10393236.32 204.38

0-20001-0-110350-20001-0-10393  234.36204.25

0-20001-0-110350-20001-0-10393  234.8204.23

0-20001-0-110350-20001-0-10393  235.33204.22

0-20001-0-110350-20001-0-10393  204.13234.6

0-20001-0-103930-20001-0-11035  204.16234.04

0-20001-0-103930-20001-0-11035 204.13 
233.830-20001-0-10393
 0-20001-0-11035203.96 
0-20001-0-10393234.63 
203.870-20001-0-11035
 0-20001-0-10393234.79 
203.75
0-20001-0-110350-20001-0-10393  234.64203.47

0-20001-0-110350-20001-0-10393  203.27234.15

0-20001-0-103930-20001-0-11035  203.19
233.950-20001-0-10393
 0-20001-0-11035203.04 
232.990-20001-0-10393
 0-20001-0-11035202.91 
232.810-20001-0-10393
 0-20001-0-11035203.06 232.38

0-20001-0-103930-20001-0-11035  203.17231.54

0-20001-0-103930-20001-0-11035  203.23231.84

0-20001-0-103930-20001-0-11035  203.37